In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from spatialdata import read_zarr

In [ ]:
sdata = read_zarr('/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep1/raw_sdata.zarr')
sdata

In [ ]:
areas = []
for pg in sdata.shapes["MultiModel"]["geometry"]:
    areas.append(pg.area)

In [ ]:
adata = sdata.tables['table'].copy()

In [ ]:
adata.obs['area'] = areas
adata.obs

In [ ]:
NP_Pad = pd.read_csv("/media/duan/DuanLab_Data/openFISH/ABA110_Demo/NP_Fold_V58.csv", index_col=0)

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
sc.set_figure_params(figsize = (6,6), dpi = 100)
sc.pl.embedding(adata, basis = 'spatial', color = 'area', size = 10)

In [ ]:
tmpmatrix = adata.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata.layers['counts'] = adata.X.copy()
adata.X = csr_matrix(tmpmatrix)
adata.layers['counts_corrected'] = adata.X.copy()

In [ ]:
adata

In [ ]:
adata.var["FP"] = adata.var_names.str.startswith("sFP")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None,qc_vars=['FP']
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", 'pct_counts_FP'],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.max(adata.obs['total_counts_FP'])

In [ ]:
adata.obs['total_counts_FP'].mean()

In [ ]:
# Filter cells according to Cell Volume
adata.obs["area"].mean()

In [ ]:
np.percentile(adata.obs["area"], 1)

In [ ]:
np.percentile(adata.obs["area"], 90)

In [ ]:
np.percentile(adata.obs["area"], 50)

In [ ]:
adata = adata[(adata.obs["area"] > 300) & (adata.obs["area"] < (3 * adata.obs["area"].mean())),:].copy()
adata

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata = adata[adata.obs.pct_counts_FP < 0.1, :].copy()
sc.pp.filter_cells(adata, min_genes=5)
sc.pp.filter_cells(adata, max_counts=1000)
sc.pp.filter_cells(adata, min_counts=20)

In [ ]:
adata

In [ ]:
adata.var_names

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
BLACK_LIST = ['sFP']

In [ ]:
adata = adata[:, [x for x in adata.var_names if x not in BLACK_LIST]].copy()

In [ ]:
adata

In [ ]:
# Normalize using cell area
from scipy.sparse import csr_matrix
cell_area = np.array(adata.obs["area"])
# cell_area = cell_area.reshape((len(cell_area),1))
adata.X = csr_matrix((adata.X.T / cell_area).T)

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=109)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="Slc17a7")

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, resolution=2, flavor="igraph",n_iterations=-1)

In [ ]:
sc.pl.umap(adata, color = 'leiden')

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="leiden")

In [ ]:
adata

In [ ]:
adata.layers['norm'] = adata.X.copy()

In [ ]:
adata.write_h5ad("./Rep1/filtered.h5ad")

In [ ]:
sdata = read_zarr('/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep2/raw_sdata.zarr')
sdata

In [ ]:
areas = []
for pg in sdata.shapes["MultiModel"]["geometry"]:
    areas.append(pg.area)

In [ ]:
adata = sdata.tables['table'].copy()

In [ ]:
adata.obs['area'] = areas
adata.obs

In [ ]:
def rotate_coordinates(coords, angle):
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    coords_array = np.array(coords)
    rotated_coords = coords_array @ rotation_matrix.T
    return rotated_coords

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
sc.set_figure_params(figsize = (5,6), dpi = 100)
sc.pl.embedding(adata, basis = 'spatial', color = 'area', size = 10)

In [ ]:
tmpmatrix = adata.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata.layers['counts'] = adata.X.copy()
adata.X = csr_matrix(tmpmatrix)
adata.layers['counts_corrected'] = adata.X.copy()

In [ ]:
adata

In [ ]:
adata.var["FP"] = adata.var_names.str.startswith("sFP")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None,qc_vars=['FP']
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", 'pct_counts_FP'],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.max(adata.obs['total_counts_FP'])

In [ ]:
adata.obs['total_counts_FP'].mean()

In [ ]:
# Filter cells according to Cell Volume
adata.obs["area"].mean()

In [ ]:
np.percentile(adata.obs["area"], 1)

In [ ]:
np.percentile(adata.obs["area"], 90)

In [ ]:
np.percentile(adata.obs["area"], 50)

In [ ]:
adata = adata[(adata.obs["area"] > 300) & (adata.obs["area"] < (3 * adata.obs["area"].mean())),:].copy()
adata

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata = adata[adata.obs.pct_counts_FP < 0.1, :].copy()
sc.pp.filter_cells(adata, min_genes=5)
sc.pp.filter_cells(adata, max_counts=1000)
sc.pp.filter_cells(adata, min_counts=20)

In [ ]:
adata

In [ ]:
adata.var_names

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
BLACK_LIST = ['sFP']

In [ ]:
adata = adata[:, [x for x in adata.var_names if x not in BLACK_LIST]].copy()

In [ ]:
adata

In [ ]:
# Normalize using cell area
from scipy.sparse import csr_matrix
cell_area = np.array(adata.obs["area"])
# cell_area = cell_area.reshape((len(cell_area),1))
adata.X = csr_matrix((adata.X.T / cell_area).T)

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=109)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="Slc17a7")

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, resolution=2, flavor="igraph",n_iterations=-1)

In [ ]:
sc.pl.umap(adata, color = 'leiden')

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="leiden", size = 7)

In [ ]:
adata

In [ ]:
adata.layers['norm'] = adata.X.copy()

In [ ]:
adata.write_h5ad("./Rep2/filtered.h5ad")

In [ ]:
import scanpy as sc
import cosg

In [ ]:
adata = sc.read_h5ad("./Rep1/filtered.h5ad")
adata

In [ ]:
groupby='leiden_R'
cosg.cosg(adata, key_added='cosg_leiden',
          # use_raw=False, layer='log1p', ## e.g., if you want to use the log1p layer in adata
          mu=1,
          expressed_pct=0.1,
          remove_lowly_expressed=True,
          n_genes_user=10,
          groupby=groupby)

import pandas as pd
pd.DataFrame(adata.uns["cosg_leiden"]["names"]).to_csv('Rep1_cosg.csv')

In [ ]:
annotation_dict = {
    '0,0': 'Fibro',
    '0,1': 'Macrophage',
    '0,2': 'SMC',
    '0,3': 'Fibro',
    '1': 'Peri',
    '10': 'Oligo',
    '11': 'Oligo',
    '12': 'Oligo',
    '13': 'Microglia',
    '14,0,0': 'Inh DMH-LHA',
    '14,0,1': 'Inh MEA-BST',
    '14,0,2': 'Inh TU-ARH',
    '14,0,3': 'Inh STR D2',
    '14,1': 'Tanycyte',
    '15': 'Ext LH-VMH',
    '16,0': 'DOUBLETs_Inh_Oligo',
    '16,1': 'Inh RT-ZI',
    '16,2': 'Unknown_SstCbln1',
    '17': 'Astro-TE',
    '18,0': 'Ext L6 CT',
    '18,1': 'Ext L6b',
    '19': 'Ext L5 CT',
    '2': 'Endo', 
    '20': 'Ext L2/3 IT',
    '21': 'Endo',
    '22': 'Ext TH',
    '23,0': 'Inh Sst',
    '23,1': 'Inh Sst',
    '23,2': 'Inh Sst',
    '24': 'Ext TH',
    '25': 'Ext MEA-COA-BMA',
    '26,0': 'Ext DMH-LHA',
    '26,1': 'Inh TU-ARH',
    '27,0': 'Ext L6 IT',
    '27,1': 'Ext L5 CT',
    '28': 'Ext L5 IT',
    '29': 'Ext L4 IT',
    '3': 'Astro-NT',
    '30,0': 'Inh Lamp5',
    '30,1': 'Ext L2/3 PIR-ENTl',
    '31': 'Ext LA-BLA-BMA-PA',
    '32': 'Ext L2/3 PIR-ENTl',
    '33': 'CHOR',
    '34': 'Inh STR D2',
    '35': 'DOUBLETs_Endo_Oligo',
    '36,0,0': 'Ext CA2',
    '36,0,1': 'Ext CA2',
    '36,0,2': 'Ext CA1',
    '36,1': 'Ext CA3',
    '36,2': 'Ext CA3',
    '37': 'Unknown_Ext',
    '38': 'Ext DG',
    '39': 'Ext CLA-EPd',
    '4': 'Astro-GLS', # glia limitans superficialis
    '40,0': 'Inh RT-ZI',
    '40,1': 'DOUBLETs_Inh_Oligo',
    '40,2': 'Unknown_Inh',
    '41': 'Ext MH',
    '42': 'Epen',
    '43': 'Inh STR D1',
    '5': 'Endo',
    '6': 'Microglia',
    '7': 'DOUBLETs_Stromal',
    '8': 'OPC',
    '9,0': 'Inh Pvalb',
    '9,1': 'Inh Vip',
    '9,2': 'Inh Reln',
    '9,3': 'Unknown_Inh'
}

In [ ]:
sc.tl.leiden(adata, restrict_to=('leiden', ['0']), resolution = 0.3)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['14']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['14,0']), resolution = 0.3)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['16']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['18']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['23']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['26']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['27']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['30']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['36']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['36,0']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['40']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['9']), resolution = 0.3)

sc.set_figure_params(figsize = (10, 10))
sc.pl.embedding(adata, basis = 'spatial', color = 'leiden_R', size = 20)

In [ ]:
sc.set_figure_params(figsize = (10, 10))
sc.pl.embedding(adata, basis = 'spatial', color = 'leiden_R', size = 20, groups=['9,3'])

In [ ]:
sc.set_figure_params(figsize=(10,5))
sc.pl.violin(adata, keys = 'Drd1', groupby='leiden_R', rotation=90)

In [ ]:
sc.set_figure_params(figsize=(10,10))
sc.pl.umap(adata, color = ['leiden_R', 'Nxph1'], size = 10, groups = ['9'])

In [ ]:
adata.obs['cell_type'] = [annotation_dict[x] for x in adata.obs['leiden_R']]

In [ ]:
sc.pl.umap(adata, color = ['cell_type'], size = 10)

In [ ]:
sc.set_figure_params(figsize = (10, 10))
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', size = 20)

In [ ]:
adata.write_h5ad("Rep1/annotated.h5ad")

In [ ]:
adata = sc.read_h5ad("./Rep2/filtered.h5ad")
adata

In [ ]:
groupby='leiden_R'
cosg.cosg(adata, key_added='cosg_leiden',
          # use_raw=False, layer='log1p', ## e.g., if you want to use the log1p layer in adata
          mu=1,
          expressed_pct=0.1,
          remove_lowly_expressed=True,
          n_genes_user=10,
          groupby=groupby)

import pandas as pd
pd.DataFrame(adata.uns["cosg_leiden"]["names"]).to_csv('Rep2_cosg.csv')

In [ ]:
annotation_dict = {
    '0,0': 'Fibro',
    '0,1': 'Peri',
    '0,2': 'Fibro',
    '0,3': 'CHOR',
    '1': 'Astro-TE',
    '10': 'Oligo',
    '11,0': 'Inh Pvalb',
    '11,1': 'Inh Pvalb',
    '11,2': 'Inh Reln',
    '11,3': 'Inh Vip',
    '12': 'Astro-NT',
    '13': 'Inh Sst',
    '14': 'Macrophage',
    '15': 'Microglia',
    '16,0': 'Inh RT-ZI',
    '16,1': 'Unknown_SstCbln1',
    '17,0': 'DOUBLETs_Ext_Inh', # 这个应该有问题
    '17,1': 'Ext L6 IT',
    '18': 'OPC',
    '19': 'Ext TH',
    '2': 'Endo',
    '20': 'DOUBLETs_Ext_Astro',
    '21,0': 'Ext LH-VMH',
    '21,1': 'Inh TU-ARH',
    '21,2': 'Ext LH-LHA',
    '21,3': 'Ext LH-LHA',
    '22': 'Ext L2/3 IT',
    '23,0,0': 'Ext L6 IT', 
    '23,0,1': 'Ext L5 CT',
    '23,1': 'Ext LA-BLA-BMA-PA',
    '24': 'Ext L2/3 RSP IT',
    '25,0': 'Ext L6 CT',
    '25,1': 'Ext L6b',
    '26,0': 'Ext L4 IT',
    '26,1': 'Ext LA-BLA-BMA-PA',
    '27,0': 'Inh DMH-LHA-MEA',
    '27,1': 'Inh DMH-LHA-MEA',
    '27,2': 'Inh DMH-LHA-MEA',
    '27,3': 'Inh PAL',
    '28,0': 'Ext L2/3 PIR-ENTl',
    '28,1': 'Ext L2/3 PIR-ENTl',
    '28,2': 'Unknown_Ext',
    '29': 'Ext CLA-EPd',
    '3': 'Endo',
    '30': 'Inh CEA',
    '31,0': 'Ext L5 IT',
    '31,1': 'Ext L5 CT',
    '32': 'Ext MEA-COA-BMA',
    '33': 'Microglia',
    '34,0': 'Ext L5 CT',
    '34,1': 'Ext L4 RSP-ACA',
    '35,0': 'Ext CA3',
    '35,1': 'Ext CA2',
    '36': 'Ext DG',
    '37': 'Epen',
    '38': 'Inh STR D2',
    '39': 'Ext CA1',
    '4': 'Endo',
    '40': 'Ext MH',
    '41': 'Ext DG',
    '42': 'Ext L2/3 PIR-ENTl',
    '43': 'Inh STR D1',
    '5': 'Oligo',
    '6': 'DOUBLETs_Peri_Oligo',
    '7': 'Oligo',
    '8,0': 'Inh RT-ZI', 
    '8,1': 'Inh RT-ZI', 
    '9': 'Ext TH'
}

In [ ]:
sc.tl.leiden(adata, restrict_to=('leiden', ['0']), resolution = 0.3)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['11']), resolution = 0.3)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['16']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['17']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['21']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['23']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['23,0']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['25']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['26']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['27']), resolution = 0.3)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['28']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['31']), resolution = 0.2)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['34']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['35']), resolution = 0.1)
sc.tl.leiden(adata, restrict_to=('leiden_R', ['8']), resolution = 0.1)

sc.set_figure_params(figsize = (8, 10))
sc.pl.embedding(adata, basis = 'spatial', color = 'leiden_R', size = 20)

In [ ]:
sc.set_figure_params(figsize = (8, 10))
sc.pl.embedding(adata, basis = 'spatial', color = 'leiden_R', size = 20, groups=['0,2'])

In [ ]:
sc.set_figure_params(figsize=(10,5))
sc.pl.violin(adata, keys = ['Tagln'], groupby='leiden_R', rotation=90)

In [ ]:
sc.set_figure_params(figsize=(10,10))
sc.pl.umap(adata, color = ['leiden_R', 'Hap1'], size = 10, groups = ['34'])

In [ ]:
adata.obs['cell_type'] = [annotation_dict[x] for x in adata.obs['leiden_R']]

In [ ]:
sc.pl.umap(adata, color = ['cell_type'], size = 10, legend_loc = 'on data')

In [ ]:
sc.set_figure_params(figsize = (7.5, 10))
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', size = 20)

In [ ]:
adata.write_h5ad("Rep2/annotated.h5ad")

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'Tagln', size = 20)

In [ ]:
df = sdata.points['dist_coord'].compute()
df

In [ ]:
counts = df['gene'].value_counts()

In [ ]:
counts.sum()

In [ ]:
NP_pad = NP_pad.set_index('Gene')

In [ ]:
result = 0
for i in NP_pad.index:
    if i != 'sFP':
        result += counts[i] * NP_pad.loc[i, 'Fold']
result

In [ ]:
cdf2 = pd.read_table("/media/duan/DuanLab_Data/openFISH/ABA110_Demo/gene_color.tsv", header=None)
cdf2

In [ ]:
from read_roi import read_roi_file

roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep2/Fig_2_roi1_big.roi')

In [ ]:
roi

In [ ]:
from shapely import Polygon
key = 'Fig_2_roi1_big'
polygon = Polygon([[roi[key]['left'], roi[key]['top']], 
                   [roi[key]['left'] + roi[key]['width'], roi[key]['top']], 
                   [roi[key]['left'] + roi[key]['width'], roi[key]['top'] + roi[key]['height']], 
                   [roi[key]['left'], roi[key]['top'] + roi[key]['height']]])
polygon

In [ ]:
from spatialdata import polygon_query
crop1 = lambda x: polygon_query(
    x,
    polygon=polygon,
    target_coordinate_system="global",
)

In [ ]:
crop1(sdata).pl.render_points('dist_coord', color = 'gene', size=0.1, palette = list(cdf2[2]), groups = list(cdf2[0])).pl.render_shapes(
    'MultiModel', outline_alpha = 1., fill_alpha = 0., outline_width = .2, outline_color = "#FFFFFF").pl.show(figsize = (20,10), dpi = 600, save = 'spots_density.pdf')

In [ ]:
adata = sc.read_h5ad("Rep2/annotated.h5ad")

In [ ]:
adata

In [ ]:
import numpy as np

In [ ]:
np.percentile(adata.obs['area'], 5)

In [ ]:
adata = adata[adata.obs['area'] > 600].copy()

In [ ]:
sc.set_figure_params(figsize = (7.5, 10), scanpy = False)
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', size = 20)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cell_type', size = 20, groups = ['Unknown_SstCbln1'])

In [ ]:
annotation_dict = {
    'Astro-NT': 'Astro-Epen',
    'Astro-TE': 'Astro-Epen',
    'CHOR': 'Astro-Epen',
    'Epen': 'Astro-Epen',
    
    'DOUBLETs_Ext_Astro': 'DOUBLETs',
    'DOUBLETs_Ext_Inh': 'DOUBLETs',
    'DOUBLETs_Peri_Oligo': 'DOUBLETs',
    
    'Endo': 'Vascular',
    'Fibro': 'Vascular',
    'Peri': 'Vascular',
    
    'Ext CA1': 'Ext HIP',
    'Ext CA2': 'Ext HIP',
    'Ext CA3': 'Ext HIP',
    'Ext DG': 'Ext HIP',
    
    'Ext CLA-EPd': 'Ext Isocortex',
    'Ext L2/3 IT': 'Ext Isocortex',
    'Ext L2/3 RSP IT': 'Ext Isocortex',
    'Ext L4 IT': 'Ext Isocortex',
    'Ext L4 RSP-ACA': 'Ext Isocortex',
    'Ext L5 CT': 'Ext Isocortex',
    'Ext L5 IT': 'Ext Isocortex',
    'Ext L6 CT': 'Ext Isocortex',
    'Ext L6 IT': 'Ext Isocortex',
    'Ext L6b': 'Ext Isocortex',
    
    'Ext LA-BLA-BMA-PA': 'Ext CTXsp',
    
    'Ext LH-LHA': 'Ext TH HY',
    'Ext LH-VMH': 'Ext TH HY',
    'Ext MH': 'Ext TH HY',
    'Ext TH': 'Ext TH HY',
    
    'Ext MEA-COA-BMA': 'Ext CNU',
    
    'Ext L2/3 PIR-ENTl': 'Ext OLF',
    
    'Inh CEA': 'Inh CNU',
    'Inh PAL': 'Inh CNU',
    'Inh STR D1': 'Inh CNU',
    'Inh STR D2': 'Inh CNU',
    
    'Inh DMH-LHA-MEA': 'Inh TH HY',
    'Inh RT-ZI': 'Inh TH HY',
    'Inh TU-ARH': 'Inh TH HY',
    
    'Inh Pvalb': 'Inh',
    'Inh Reln': 'Inh',
    'Inh Sst': 'Inh',
    'Inh Vip': 'Inh',
    
    'Macrophage': 'Immune',
    'Microglia': 'Immune',
    
    'OPC': 'Oligo-OPC',
    'Oligo': 'Oligo-OPC',
    
    'Unknown_Ext': 'Unknown',
    'Unknown_SstCbln1': 'Inh'
}

In [ ]:
adata.obs['Part'] = [annotation_dict[x] for x in adata.obs['cell_type']]

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'Part', size = 20)

In [ ]:
check = adata[~adata.obs['Part'].isin(['Unknown', 'DOUBLETs'])].copy()
check

In [ ]:
check.uns['Part_colors'] = [
    '#00FFFFff', '#8B5A2Bff', '#CD1076ff', '#00008Bff', '#1E90FFff',
    '#9B30FFff', '#2E8B57ff', '#FFFF00ff', '#00FF00ff', '#CD661Dff', '#009ACDff',
    '#696969ff', '#FF3030ff'
]

In [ ]:
sc.pl.embedding(check, basis = 'spatial', color = 'Part', size = 20)

In [ ]:
sdata = read_zarr('/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep2/raw_sdata.zarr')
sdata

In [ ]:
import spatialdata_plot

In [ ]:
sdata.shapes['MultiModel'].index = sdata.shapes['MultiModel'].index.astype('str')
sdata.shapes['MultiModel']['Part'] = check.obs['Part']
sdata.shapes['MultiModel']['Part'] = sdata.shapes['MultiModel']['Part'].astype('category')
sdata.shapes['MultiModel'] = sdata.shapes['MultiModel'].dropna()

In [ ]:
sdata.pl.render_shapes("MultiModel", color = 'Part', palette=list(check.uns['Part_colors']), groups = sorted(check.obs['Part'].unique().tolist()),
                      outline_alpha = 0., fill_alpha = 1).pl.show(title="MultiModel", dpi = 900, frameon = False, save = 'Part_Rep2.pdf')

In [ ]:
cdf = pd.read_table('Rep2_CellType_Color.tsv', header=None, index_col=0)
cdf

In [ ]:
sdata.shapes['MultiModel'].index = sdata.shapes['MultiModel'].index.astype('str')
sdata.shapes['MultiModel']['cell_type'] = check.obs['cell_type']
sdata.shapes['MultiModel']['cell_type'] = sdata.shapes['MultiModel']['cell_type'].astype('category')
sdata.shapes['MultiModel'] = sdata.shapes['MultiModel'].dropna()

In [ ]:
sdata.pl.render_shapes("MultiModel", color = 'cell_type', palette=[cdf.loc[x,1] + 'ff' for x in sorted(check.obs['cell_type'].unique().tolist())], groups = sorted(check.obs['cell_type'].unique().tolist()),
                      outline_alpha = 0., fill_alpha = 1).pl.show(figsize = (20,10), title="MultiModel", dpi = 900, frameon = False, save = 'CellType_Rep2.pdf')

In [ ]:
check

In [ ]:
from read_roi import read_roi_file

roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep2/Fig_2C_roi1.roi')

In [ ]:
roi

In [ ]:
from shapely import Polygon
import numpy as np

key = 'Fig_2C_roi1'
polygon = Polygon(

        np.column_stack([roi[key]['x'], roi[key]['y']])

)
polygon

In [ ]:
from spatialdata import polygon_query
crop1 = lambda x: polygon_query(
    x,
    polygon=polygon,
    target_coordinate_system="global",
)

In [ ]:
crop1(sdata).pl.render_shapes("MultiModel", color = 'cell_type', palette=[cdf.loc[x,1] + 'ff' for x in sorted(check.obs['cell_type'].unique().tolist())], groups = sorted(check.obs['cell_type'].unique().tolist()),
                      outline_alpha = 0, fill_alpha = 1.).pl.show(title="MultiModel", figsize = (20,10), dpi = 150, frameon = False, save = 'demo_roi1_HIP.pdf')

In [ ]:
from read_roi import read_roi_file

roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep2/Fig_2C_roi2.roi')

In [ ]:
roi

In [ ]:
from shapely import Polygon
import numpy as np

key = 'Fig_2C_roi2'
polygon = Polygon(

        np.column_stack([roi[key]['x'], roi[key]['y']])

)
polygon

In [ ]:
from spatialdata import polygon_query
crop1 = lambda x: polygon_query(
    x,
    polygon=polygon,
    target_coordinate_system="global",
)

In [ ]:
crop1(sdata).pl.render_shapes("MultiModel", color = 'cell_type', palette=[cdf.loc[x,1] + 'ff' for x in sorted(check.obs['cell_type'].unique().tolist())], groups = sorted(check.obs['cell_type'].unique().tolist()),
                      outline_alpha = 0, fill_alpha = 1.).pl.show(title="MultiModel",figsize = (10,20), dpi = 150, frameon = False, save = 'demo_roi2_Ventricle.pdf')

In [ ]:
from read_roi import read_roi_file

roi = read_roi_file('/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep2/Fig_2C_roi3.roi')

In [ ]:
roi

In [ ]:
from shapely import Polygon
import numpy as np

key = 'Fig_2C_roi3'
polygon = Polygon(

        np.column_stack([roi[key]['x'], roi[key]['y']])

)
polygon

In [ ]:
from spatialdata import polygon_query
crop1 = lambda x: polygon_query(
    x,
    polygon=polygon,
    target_coordinate_system="global",
)

In [ ]:
groups = sorted(check.obs['cell_type'].unique().tolist())
groups

In [ ]:
pallette = [cdf.loc[x,1] + 'ff' for x in sorted(check.obs['cell_type'].unique().tolist())]

In [ ]:
pallette = np.array(pallette)
groups = np.array(groups)

In [ ]:
groups

In [ ]:
crop1(sdata).pl.render_shapes("MultiModel", color = 'cell_type', palette=list(pallette[[0,1,3,8,10,13,15,16,17,18,19,20,26,42]]) , groups = list(groups[[0,1,3,8,10,13,15,16,17,18,19,20,26,42]]),
                      outline_alpha = 0, fill_alpha = 1.).pl.show(title="MultiModel", figsize = (20,10), dpi = 150, frameon = False, save = 'demo_roi3_CortexExt.pdf')

In [ ]:
groups[[30,32,35,37]]

In [ ]:
crop1(sdata).pl.render_shapes("MultiModel", color = 'cell_type', palette=list(pallette[[30,32,35,37,43,26,0,1,3,42]]) , groups = list(groups[[30,32,35,37,43,26,0,1,3,42]]),
                      outline_alpha = 0, fill_alpha = 1.).pl.show(title="MultiModel", figsize = (20,10), dpi = 150, frameon = False, save = 'demo_roi3_CortexInh.pdf')

In [ ]:
check

In [ ]:
sc.set_figure_params(figsize = (5, 5), scanpy = False, dpi_save=300)
sc.pl.umap(check, color = 'Part', size = 10, save='_Part.pdf')

In [ ]:
sc.set_figure_params(figsize = (5, 5), scanpy = False, dpi_save=300)
sc.pl.umap(check, color = 'cell_type', size = 10, save='_CellType.pdf')

In [ ]:
check.uns['cell_type_colors'] = pallette

In [ ]:
import scanpy as sc

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig,ax = plt.subplots(figsize = (3,4))
sns.violinplot(check.obs['total_counts'], ax = ax, color = '#FFFFFF', inner = 'box', linecolor="#000000")
plt.savefig("./figures/Rep2_total_counts.pdf", dpi = 150)
plt.show()

In [ ]:
check.obs['total_counts'].median()

In [ ]:
check.obs['n_genes_by_counts'].median()

In [ ]:
fig,ax = plt.subplots(figsize = (3,4))
sns.violinplot(check.obs['n_genes_by_counts'], ax = ax, color = '#FFFFFF', inner = 'box', linecolor="#000000")
plt.savefig("./figures/Rep2_n_genes.pdf", dpi = 150)
plt.show()

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.read_h5ad("/media/duan/DuanLab_Data/openFISH/ABA110_Demo2/Rep2/annotated.h5ad")
adata

In [ ]:
check = adata[~adata.obs['Part'].isin(['Unknown', 'DOUBLETs'])].copy()
check

In [ ]:
check.obs['pct_counts_FP'].median()

In [ ]:
check.obs['pct_counts_FP'].max()